In [3]:
!pipenv install browsermob-proxy

Installing browsermob-proxy...

Installing dependencies from Pipfile.lock (15d44e)...


Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 to suppress this warning.


[    ] Installing..
[=   ] Installing browsermob-proxy..
[==  ] Installing browsermob-proxy..
[=== ] Installing browsermob-proxy..
[ ===] Installing browsermob-proxy..
[  ==] Installing browsermob-proxy..
[   =] Installing browsermob-proxy..
[    ] Installing browsermob-proxy..
[   =] Installing browsermob-proxy..
[  ==] Installing browsermob-proxy..
[ ===] Installing browsermob-proxy..
[====] Installing browsermob-proxy..
[=== ] Installing browsermob-proxy..
[==  ] Installing browsermob-proxy..
[=   ] Installing browsermob-proxy..
[    ] Installing browsermob-proxy..
[=   ] Installing browsermob-proxy..
[==  ] Installing browsermob-proxy..
[=== ] Instal

In [2]:
! pipenv install selenium-wire

Installing selenium-wire...



Courtesy Notice: Pipenv found itself running within a virtual environment, so it will automatically use that environment, instead of creating its own for any project. You can set PIPENV_IGNORE_VIRTUALENVS=1 to force pipenv to ignore that environment and create its own instead. You can set PIPENV_VERBOSITY=-1 to suppress this warning.


[    ] Installing..
[=   ] Installing selenium-wire..
[==  ] Installing selenium-wire..
[=== ] Installing selenium-wire..
[ ===] Installing selenium-wire..
[  ==] Installing selenium-wire..
[   =] Installing selenium-wire..
[    ] Installing selenium-wire..
[   =] Installing selenium-wire..
[  ==] Installing selenium-wire..
[ ===] Installing selenium-wire..
[====] Installing selenium-wire..
[=== ] Installing selenium-wire..
[==  ] Installing selenium-wire..
[=   ] Installing selenium-wire..
[    ] Installing selenium-wire..
[=   ] Installing selenium-wire..
[==  ] Installing selenium-wire..
[=== ] Installing selenium-wire..
[ ===] Installing selenium-wir

In [3]:
import configparser
from seleniumwire import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.keys import Keys

import time
import pandas as pd
import os

# Get the current directory
current_dir = os.getcwd()

# Construct the relative path to the download directory
download_dir = os.path.join(current_dir, 'data')

# Construct the relative path to the chromedriver executable
driver_path = os.path.join(current_dir, 'chromedriver.exe')

# Configure any desired options
options = Options()
options.add_experimental_option("prefs", {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})

# Create the webdriver instance with seleniumwire
seleniumwire_options = {
    'connection_timeout': None,  # Disable connection timeout
    'verify_ssl': False  # Disable SSL verification
}
service = Service(driver_path)
browser = webdriver.Chrome(service=service, seleniumwire_options=seleniumwire_options, options=options)

browser.get('https://portal.erc.go.ke:8446/site/login')

# Load the configuration file
config = configparser.ConfigParser()
config.read('config.ini')

# Retrieve the username and password from the configuration file
username = config.get('Credentials', 'username')
password = config.get('Credentials', 'password')

username_element = browser.find_element(By.NAME, 'LoginForm[username]')
password_element = browser.find_element(By.NAME, 'LoginForm[password]')

username_element.send_keys(username)
password_element.send_keys(password)
time.sleep(15)
login_button = browser.find_element(By.NAME, 'login-button')
login_button.click()

# Navigating to the electricity department menu
time.sleep(10)
main_menu = browser.find_element(By.XPATH, '/html/body/div[2]/header/nav/div/div/div[2]/ul/li[7]')
main_menu.click()
sub_menu = browser.find_element(By.XPATH, '/html/body/div[2]/header/nav/div/div/div[2]/ul/li[7]/ul/li[1]')
sub_menu.click()

# Clicking the Excel Button
excel_button = browser.find_element(By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div[1]/button[1]')
excel_button.click()
time.sleep(5)

# Load the Excel file
df = pd.read_excel('C:/Users/ip 5/mike/work_related/data/ERC Internal Portal.xlsx')
data = {"Links": []}
df_data = pd.DataFrame()

# Loop through the rows of the column containing the 'review?ref=' strings
# Import the dictionary from the xpaths module
from list_of_xpath import electrical_power_undertaking, contractor_xpaths, electrician_xpaths

for row1, row2 ,row3 in zip(df.iloc[2:, 0], df.iloc[2:, 1],df.iloc[2:,4]):
    # Split the string at the equal sign to extract the value
    my_string = 'https://portal.erc.go.ke:8446/new-licence-applications/review?ref='
    my_integer = row1

    # Use an f-string to insert the integer into the string
    result = f'{my_string}{my_integer}'
    data["Links"].append(result)
    browser.get(result)
    tabs = browser.find_elements(By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div[2]/div/ul/li')
    applicant_data = {}

    for tab in tabs:
        tab.click()
        time.sleep(2)
        # Define the root XPATH
        root_xpath = '/html/body/div[2]/div/div/div[2]/div/div[2]/div/div/div[2]/div/form'
        # Locate all the input elements that are children of div elements under the root XPATH
        input_elements = browser.find_elements(By.XPATH, f'{root_xpath}//div/input')
        # Define a dictionary to store the extracted data for the current tab
        tab_data = {}

        # Loop through the located input elements
        for i, input_element in enumerate(input_elements):
            # Extract the value of the current input element
            input_value = input_element.get_attribute('value')
            # Store the extracted value in the dictionary using a column name based on the index of the input element
            tab_data[f'Input {i + 1}'] = input_value

        # Append the extracted data for the current tab to the DataFrame
        df_data = pd.concat([df_data, pd.DataFrame(tab_data, index=[0])], ignore_index=True)

        # Check if the current tab is the "documents" tab
        if tab.text.strip() == "Documents":
            # Extract the applicant number and applicant name
            applicant_number = row1
            applicant_name = row2

            # Modify the folder name
            folder_name = f"{applicant_name.replace(' ', '_')}_{applicant_number}"

            # Create the folder if it doesn't exist
            folder_path = os.path.join(download_dir, folder_name)
            os.makedirs(folder_path, exist_ok=True)

            # Click the button to download the documents
            documents_button = browser.find_element(By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div[2]/div/div/div[14]/h4/a')
            documents_button.click()

            # Wait for the documents to be downloaded (adjust the wait time if needed)
            time.sleep(5)

            # Move the downloaded documents to the applicant's folder
            files = os.listdir(download_dir)
            for file in files:
                if file.endswith('.pdf'):
                    src_path = os.path.join(download_dir, file)
                    dst_path = os.path.join(folder_path, file)
                    os.rename(src_path, dst_path)

    df_company = pd.DataFrame(data)
    df_company.to_excel("output_1.xlsx", index=False)
    df_data.to_excel('input_values.xlsx', index=False)

browser.quit()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="LoginForm[username]"]"}
  (Session info: chrome=113.0.5672.127)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x003CDCE3+50899]
	(No symbol) [0x0035E111]
	(No symbol) [0x00265588]
	(No symbol) [0x002908F9]
	(No symbol) [0x00290AFB]
	(No symbol) [0x002BF902]
	(No symbol) [0x002AB944]
	(No symbol) [0x002BE01C]
	(No symbol) [0x002AB6F6]
	(No symbol) [0x00287708]
	(No symbol) [0x0028886D]
	GetHandleVerifier [0x00633EAE+2566302]
	GetHandleVerifier [0x006692B1+2784417]
	GetHandleVerifier [0x0066327C+2759788]
	GetHandleVerifier [0x00465740+672048]
	(No symbol) [0x00368872]
	(No symbol) [0x003641C8]
	(No symbol) [0x003642AB]
	(No symbol) [0x003571B7]
	BaseThreadInitThunk [0x76597D59+25]
	RtlInitializeExceptionChain [0x776EB74B+107]
	RtlClearBits [0x776EB6CF+191]


In [4]:
import configparser
from seleniumwire import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.common.keys import Keys

import time
import pandas as pd
import os

# Get the current directory
current_dir = os.getcwd()

# Construct the relative path to the download directory
download_dir = os.path.join(current_dir, 'data')

# Construct the relative path to the chromedriver executable
driver_path = os.path.join(current_dir, 'chromedriver.exe')

# Configure any desired options
options = Options()
options.add_experimental_option("prefs", {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})

# Create the webdriver instance with seleniumwire
seleniumwire_options = {
    'connection_timeout': None,  # Disable connection timeout
    'verify_ssl': False  # Disable SSL verification
}
service = Service(driver_path)
browser = webdriver.Chrome(service=service, seleniumwire_options=seleniumwire_options, options=options)

browser.get('https://portal.erc.go.ke:8446/site/login')

# Load the configuration file
config = configparser.ConfigParser()
config.read('config.ini')

# Retrieve the username and password from the configuration file
username = config.get('Credentials', 'username')
password = config.get('Credentials', 'password')

username_element = browser.find_element(By.NAME, 'LoginForm[username]')
password_element = browser.find_element(By.NAME, 'LoginForm[password]')

username_element.send_keys(username)
password_element.send_keys(password)
time.sleep(15)
login_button = browser.find_element(By.NAME, 'login-button')
login_button.click()

# Navigating to the electricity department menu
time.sleep(10)
main_menu = browser.find_element(By.XPATH, '/html/body/div[2]/header/nav/div/div/div[2]/ul/li[7]')
main_menu.click()
sub_menu = browser.find_element(By.XPATH, '/html/body/div[2]/header/nav/div/div/div[2]/ul/li[7]/ul/li[1]')
sub_menu.click()

# Clicking the Excel Button
excel_button = browser.find_element(By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div[1]/button[1]')
excel_button.click()
time.sleep(5)

# Load the Excel file
df = pd.read_excel('C:/Users/ip 5/mike/work_related/data/ERC Internal Portal.xlsx')
data = {"Links": []}
df_data = pd.DataFrame()
df_undertaking = pd.DataFrame()  # DataFrame for Electric Power Undertaking
df_contractor = pd.DataFrame()  # DataFrame for Electrical Contractor
df_electrician = pd.DataFrame()  # DataFrame for Electrician


# Loop through the rows of the column containing the 'review?ref=' strings
# Import the dictionary from the xpaths module
from list_of_xpath import electrical_power_undertaking, contractor_xpaths, electrician_xpaths

for row1, row2 ,row3 in zip(df.iloc[2:, 0], df.iloc[2:, 1],df.iloc[2:,4]):
    # Split the string at the equal sign to extract the value
    my_string = 'https://portal.erc.go.ke:8446/new-licence-applications/review?ref='
    my_integer = row1

    # Use an f-string to insert the integer into the string
    result = f'{my_string}{my_integer}'
    data["Links"].append(result)
    browser.get(result)
    tabs = browser.find_elements(By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div[2]/div/ul/li')
    applicant_data = {}

     
    for tab in tabs:
        tab_title = tab.text.strip()
        
        # Check if the tab title matches the desired category
        if row3 == 'Electric Power Undertaking' and tab_title in electrical_power_undertaking:
            tab_xpaths = electrical_power_undertaking[tab_title]
            df_category = df_undertaking  # Use the DataFrame for Electric Power Undertaking
        elif row3 == 'Electrical Contractor' and tab_title in contractor_xpaths:
            tab_xpaths = contractor_xpaths[tab_title]
            df_category = df_contractor  # Use the DataFrame for Electrical Contractor
        elif row3 == 'Electrician' and tab_title in electrician_xpaths:
            tab_xpaths = electrician_xpaths[tab_title]
            df_category = df_electrician  # Use the DataFrame for Electrician
        else:
            continue
        
        # Click the tab and extract the data using the matched xpath
        tab.click()
        time.sleep(2)
        tab_data = {}
        
        for field, xpath in tab_xpaths.items():
            input_element = browser.find_element(By.XPATH, xpath)
            input_value = input_element.get_attribute('value')
            tab_data[field] = input_value
        

        
        # Append the extracted data for the current tab to the DataFrame
        df_data = pd.concat([df_data, pd.DataFrame(tab_data, index=[0])], ignore_index=True)

        
    df_company = pd.DataFrame(data)
    df_company.to_excel("output_1.xlsx", index=False)
    df_data.to_excel('input_values.xlsx', index=False)

    df_undertaking.to_excel('electric_power_undertaking.xlsx', index=False)
    df_contractor.to_excel('electrical_contractor.xlsx', index=False)
    df_electrician.to_excel('electrician.xlsx', index=False)
browser.quit()


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[name="LoginForm[username]"]"}
  (Session info: chrome=114.0.5735.110)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00DCA813+48355]
	(No symbol) [0x00D5C4B1]
	(No symbol) [0x00C65358]
	(No symbol) [0x00C909A5]
	(No symbol) [0x00C90B3B]
	(No symbol) [0x00CBE232]
	(No symbol) [0x00CAA784]
	(No symbol) [0x00CBC922]
	(No symbol) [0x00CAA536]
	(No symbol) [0x00C882DC]
	(No symbol) [0x00C893DD]
	GetHandleVerifier [0x0102AABD+2539405]
	GetHandleVerifier [0x0106A78F+2800735]
	GetHandleVerifier [0x0106456C+2775612]
	GetHandleVerifier [0x00E551E0+616112]
	(No symbol) [0x00D65F8C]
	(No symbol) [0x00D62328]
	(No symbol) [0x00D6240B]
	(No symbol) [0x00D54FF7]
	BaseThreadInitThunk [0x75937D59+25]
	RtlInitializeExceptionChain [0x771CB74B+107]
	RtlClearBits [0x771CB6CF+191]
